# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [87]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
import sys
key = "C:\\Users\\Kelly\\Desktop\\KEYS"
sys.path.append(key)
from GoogleAPI import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [88]:
city_df = pd.read_csv('clean_city_data.csv')
city_df

,Unnamed: 0,City ID,City Name,Cloudiness,Wind Speed,Temperature,Feels Like,Min Temp,Max Temp,Barometric Pressure,Humidity,Sea Level,Ground Level,Latitude,Longitude
0,0,2685750,Östersund,99.0,100.00,2721.0,267.45,272.04,272.15,1013.0,100.0,NaN,NaN,NaN,NaN
1,1,3831208,Qaanaaq,100.0,59.00,26381.0,258.75,263.81,263.81,1010.0,81.0,1010.0,1010.0,86.349741,82.118957
2,2,5554428,Ketchikan,1.0,60.00,27415.0,269.83,274.15,274.15,1011.0,80.0,NaN,NaN,-52.936276,16.030378
3,3,2206939,Bluff,56.0,11.62,28537.0,280.11,285.37,285.37,1025.0,80.0,NaN,NaN,-32.749052,85.423768
4,4,1525798,Balkhash,0.0,70.00,25215.0,244.26,252.15,252.15,1037.0,83.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,558,1805518,Jining,0.0,6.00,27508.0,266.40,275.08,275.08,1031.0,69.0,1031.0,1026.0,NaN,NaN
559,559,1856434,Murakami,75.0,360.00,27915.0,270.62,279.15,279.15,1013.0,87.0,NaN,NaN,NaN,NaN
560,560,2171069,Colac,100.0,5.81,29935.0,298.82,297.04,300.93,1017.0,45.0,NaN,NaN,NaN,NaN
561,561,3698608,Celendín,100.0,118.00,28458.0,284.25,284.58,284.58,1016.0,92.0,1016.0,748.0,NaN,NaN


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [89]:
gmaps.configure(api_key=gkey)
city_df = city_df.dropna()
locations = city_df[["Latitude", "Longitude"]].astype(float)

weights = city_df["Humidity"].astype(float)


In [90]:
# fig = gmaps.figure()

# heat_layer = gmaps.heatmap_layer(locations, weights, 
#                                  dissipating=False, max_intensity=100,
#                                  point_radius = 4)

# fig.add_layer(heat_layer)

# fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [91]:
ideal_weather = city_df.loc[city_df['Humidity'] < 50]
ideal_weather = city_df.loc[city_df['Humidity'] > 30]
ideal_weather = city_df.loc[city_df['Feels Like'] < 1026.28]
ideal_weather = city_df.loc[city_df['Feels Like'] > 1005.76]
ideal_weather = city_df.loc[city_df['Max Temp'] < 296.5]
ideal_weather = city_df.loc[city_df['Min Temp'] > 291.5]
ideal_weather = city_df.loc[city_df['Wind Speed'] < 24]
ideal_weather = city_df.loc[city_df['Cloudiness'] > 40]
dropped_weather = ideal_weather.dropna(subset=['Humidity', 'Feels Like', 'Max Temp', 'Min Temp', 'Wind Speed', 'Cloudiness', 'Latitude', 'Longitude'])
hotel_checker = dropped_weather.sample(10)
hotel_checker

,Unnamed: 0,City ID,City Name,Cloudiness,Wind Speed,Temperature,Feels Like,Min Temp,Max Temp,Barometric Pressure,Humidity,Sea Level,Ground Level,Latitude,Longitude
244,244,3412093,Vestmannaeyjar,94.0,90.0,27966.0,269.72,279.66,279.66,994.0,86.0,994.0,993.0,63.191140,104.055993
55,55,2063036,Port Lincoln,100.0,36.0,29467.0,290.75,294.67,294.67,1008.0,79.0,1008.0,1005.0,-4.557161,18.317279
512,512,3665210,Rodrigues Alves,49.0,340.0,29484.0,298.92,294.84,294.84,1012.0,98.0,1012.0,991.0,-0.236168,59.509967
171,171,1651810,Airai,100.0,278.0,29395.0,296.96,293.95,293.95,1008.0,94.0,1008.0,855.0,54.017123,53.724426
467,467,2013279,Vostok,100.0,266.0,25559.0,250.02,255.59,255.59,1015.0,89.0,1015.0,926.0,-65.462931,-10.016789
271,271,2125693,Evensk,100.0,352.0,25848.0,251.77,258.48,258.48,1017.0,86.0,1017.0,1010.0,-76.673095,154.728901
372,372,2121025,Srednekolymsk,100.0,261.0,24285.0,236.76,242.85,242.85,1031.0,87.0,1031.0,1028.0,12.874629,135.396047
123,123,3372707,Ribeira Grande,51.0,284.0,28917.0,280.97,289.17,289.17,1017.0,68.0,1017.0,1014.0,48.249229,-36.761909
103,103,2022572,Khatanga,100.0,145.0,24647.0,240.38,246.47,246.47,1005.0,89.0,1005.0,1002.0,20.763353,12.615508
214,214,2126710,Beringovskiy,100.0,33.0,26682.0,256.83,266.82,266.82,1002.0,97.0,1002.0,1000.0,59.267785,179.496269


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [92]:
hotel_df = hotel_checker.reindex(columns = ["City Name", "Latitude", "Longitude"])
hotel_df["Hotel Name"] = ""
hotel_df = hotel_df.reset_index().rename(columns={"City Name":"City"})
hotel_df

,index,City,Latitude,Longitude,Hotel Name
0,244,Vestmannaeyjar,63.191140,104.055993,
1,55,Port Lincoln,-4.557161,18.317279,
2,512,Rodrigues Alves,-0.236168,59.509967,
3,171,Airai,54.017123,53.724426,
4,467,Vostok,-65.462931,-10.016789,
5,271,Evensk,-76.673095,154.728901,
6,372,Srednekolymsk,12.874629,135.396047,
7,123,Ribeira Grande,48.249229,-36.761909,
8,103,Khatanga,20.763353,12.615508,
9,214,Beringovskiy,59.267785,179.496269,


In [93]:
# #the latitudes and longitudes I saved from citipy were not getting along with the google api at this point, so I decided
#     #to get the place_id from the api and then get google's preferred coordinates from their own native api in order to 
#     #get to the hotel data.

# #get place_id for city
# placeid_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
# # use iterrows to iterate through pandas dataframe
# for index, row in hotel_df.iterrows():
#         placeid_params = {
#         "input": row['City'],
#         "inputtype": "textquery",
#         "key": gkey,
#     }
#         ID = requests.get(placeid_url, params=placeid_params).json()
#         ID = (ID["candidates"][0]["place_id"])
#         print(ID)
    

In [94]:
# #get coordinates from place_id
# coordinate_url = "https://maps.googleapis.com/maps/api/place/details/json?"
# # use iterrows to iterate through pandas dataframe
# for index, row in hotel_df.iterrows():
#         coordinate_params = {
#         "place_id": ID,
#         "key": gkey,
#     }
# #     assemble url and make API request
#         coords = requests.get(coordinate_url, params=coordinate_params).json()
#         lat = coords['result']['geometry']['location']['lat']
#         lng = coords['result']['geometry']['location']['lng']
#         coords = f'{lat},{lng}'
#         print(coords)

In [95]:
# find the closest restaurant of each type to coordinates


restaurant_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    #get place_id for city name
    placeid_params = {
        "input": row['City'],
        "inputtype": "textquery",
        "key": gkey,
    }
    ID = requests.get(placeid_url, params=placeid_params).json()
    ID = (ID["candidates"][0]["place_id"])
    City = row['City']
    restaurant_params = {
        "location": coords,  
        "radius": 5000,
        "types": "lodging",
        "key": gkey,
    }
    #use place_id to get coordinates that agree with google's api
    coordinate_params = {
        "place_id": ID,
        "key": gkey,
    }
#     assemble url and make API request
    coords = requests.get(coordinate_url, params=coordinate_params).json()
    lat = coords['result']['geometry']['location']['lat']
    lng = coords['result']['geometry']['location']['lng']
#    Country = coords['result']['address_components'][3]
    coords = f'{lat},{lng}'
#    print(Country)

#     assemble url and make API request
    print(f"Retrieving Results for hotels near {City}:")
    response = requests.get(restaurant_url, params=restaurant_params).json()
    # extract results
    results = response['results']
    
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
    except (KeyError, IndexError):
        print("Sorry, there are no hotels within 5000 meters of this city. Maybe try someplace less remote?")
        
    print("------------")

Retrieving Results for hotels near Vestmannaeyjar:
Sorry, there are no hotels within 5000 meters of this city. Maybe try someplace less remote?
------------
Retrieving Results for hotels near Port Lincoln:
Closest hotel is Glamping & Camping.
------------
Retrieving Results for hotels near Rodrigues Alves:
Closest hotel is Port Lincoln Hotel.
------------
Retrieving Results for hotels near Airai:
Closest hotel is Pousada Aconchego.
------------
Retrieving Results for hotels near Vostok:
Sorry, there are no hotels within 5000 meters of this city. Maybe try someplace less remote?
------------
Retrieving Results for hotels near Evensk:
Closest hotel is easyHotel Berlin Hackescher Markt.
------------
Retrieving Results for hotels near Srednekolymsk:
Sorry, there are no hotels within 5000 meters of this city. Maybe try someplace less remote?
------------
Retrieving Results for hotels near Ribeira Grande:
Sorry, there are no hotels within 5000 meters of this city. Maybe try someplace less re

In [96]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Country'

In [ ]:
# Add marker layer ontop of heat map


# Display figure
